In [84]:
%pylab notebook
from spi_helper import *
sys.path.append("../..")
from common import *

Populating the interactive namespace from numpy and matplotlib


# Connec to FPGA

Start remote litex server first
```bash
make server
```

In [91]:
r = conLitexServer('../build/csr.csv')

Connected to Port 1234
AD9174 + VC707 test 2020-03-10 17:12:08


# Play with GTX phy

In [93]:
r.regs.gtphy_transmitter_produce_square_wave.write(0)
r.regs.gtphy_transmitter_produce_square_wave.read()

0

In [59]:
r.regs.gtphy_transmitter_txdiffcttrl.write(15)
r.regs.gtphy_transmitter_txdiffcttrl.read()

15

In [63]:
r.regs.gtphy_transmitter_txpostcursor.write(0)
r.regs.gtphy_transmitter_txpostcursor.read()

0

In [65]:
r.regs.gtphy_transmitter_txprecursor.write(0)
r.regs.gtphy_transmitter_txprecursor.read()

0

In [34]:
r.regs.gtphy_transmitter_txmaincursor.write(80)
r.regs.gtphy_transmitter_txmaincursor.read()

80

# Setup AD9174

In [94]:
ad = AdSpi(r)

In [106]:
# Power up sequence, Table 51
ad.wr(0x000, 0x81)  # Soft reset
ad.wr(0x000, 0x3C)  # 4 - wire SPI mode + ADDRINC
ad.wr(0x091, 0x00)  # Power up clock RX
ad.wr(0x206, 0x01)  # Enable PHYs
ad.wr(0x705, 0x01)  # Enable boot loader
ad.wr(0x090, 0x00)  # Power on DACs and bias supply

# Print product ID
print(hex(ad.rr(0x004)))
print(hex(ad.rr(0x005)))

0x74
0x91


In [107]:
# Disable DAC PLL and config for external clock, Table 52
ad.wr(0x095, 0x01)
ad.wr(0x790, 0xFF)
ad.wr(0x791, 0xFF)

ad.wr(0x008, (1 << 7) | (1 << 6)) # Select both DACs

# Magic numbers from Table 54 (calibration)
ad.wr(0x050, 0x2A)
ad.wr(0x061, 0x68)
ad.wr(0x051, 0x82)
ad.wr(0x051, 0x83)
print('CAL_STAT:', ad.rr(0x052))
ad.wr(0x081, 0x03)  # Power down calibration clocks

# JESD config, Table 55
ad.wr(0x100, 0x00)  # Power up digital datapath clocks
ad.wr(0x201, 0xFF)  # Power down unused PHYs.

ADC_CLK_DIVIDER = 6
# ADC clock output divider = /4
#   0: Divide by 1
#   1: Divide by 2
#   2: Divide by 3
#   3: Divide by 4
ad.wr(0x799, (3 << ADC_CLK_DIVIDER))

CAL_STAT: 1


0

# Setup HMC7044
  * External VCO input
  * clkout12 = f_vco / 10 = 128 MHz

In [109]:
hmc = HmcSpi(r)

hmc.wr(0x000, 1)  # reset
hmc.wr(0x000, 0)

hmc.wr(0x054, 0)  # Disable SDATA driver (uni-direct. buffer)
hmc.wr(0x001, (1 << 6) | (1 << 5)) # High performance dividers / PLL

# VCO Selection
# 0 Internal disabled/external
# 1 High
# 2 Low
VCO_SELECT = 3
hmc.wr(0x003, (0 << VCO_SELECT))

# Enable output channel 12 and 13
hmc.wr(0x004, (1 << (12 // 2)))

# clkin1 as external VCO input
hmc.wr(0x005, (1 << 5))

# Magic numbers from datasheet Table 74
hmc.wr(0x09F, 0x4D)
hmc.wr(0x0A0, 0xDF)
hmc.wr(0x0A5, 0x06)
hmc.wr(0x0A8, 0x06)
hmc.wr(0x0B0, 0x04)

hmc.setupChannel(12, 10)

In [108]:
hmc.wr(0x000, 1)  # reset
hmc.wr(0x000, 0)

In [ ]:
r.close()